In [ ]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')
model.eval()
inputs = processor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_states = outputs[0]
print(inputs["pixel_values"].shape)
print(last_hidden_states.shape)

# We have to force return_dict=False for tracing
model.config.return_dict = False

with torch.no_grad():
    traced_model = torch.jit.trace(model, [inputs.pixel_values])
    traced_outputs = traced_model(inputs.pixel_values)
print("==============================================")
print((last_hidden_states - traced_outputs[0]).abs().max())
print("==============================================")

In [49]:
images = [Image.open("../../img/cat.png"),Image.open("../../img/cat.jpg")]
inputs = processor(images=images,return_tensors="pt")
outputs = model(**inputs,output_hidden_states=False)
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([2, 257, 768])
torch.Size([2, 768])


In [48]:
outputs[0][1:,:].shape

torch.Size([1, 257, 768])

In [51]:
torch.mean(torch.eq(outputs[0][:,0,:],outputs[1]).float())

tensor(1.)

outputs[0][:,0,:] and outputs[1] both represent the class token